# OFDM

This notebook focuses on Orthogonal Frequency Division Multiplexing (OFDM). It provides an example implementation of a basic OFDM transmitter and receiver. We analyze the spectral properties of an OFDM signal and demonstrate that the symbol error rate in AWGN is identical to the theoretical limit. For a diespersive channel, we show that the cyclic prefix effectively removes inter-symbol interference.

<a href="https://colab.research.google.com/github/bepepa/digital_comms/blob/master/040_ofdm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> (Executable notebook)

<a href="https://nbviewer.org/format/slides/github/bepepa/digital_comms/blob/main/040_ofdm.ipynb"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" alt="Render nbviewer" /> </a> (read-only, slides view)

In [1]:
## Boilerplate instructions for importing NumPy and Matplotlib
# Import NumPy
import numpy as np

# To plot pretty figures, use matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## OFDM Transmitter and Receiver

An OFDM signal is a multi-carrier signal. Each OFDM subcarrier supports a narrowband, linearly modulated signal. We will assume that the information to be transmitted consists of a length-$L$ sequence of symbols $s_n$. To generate a baseband OFDM signal, the following steps are performed.


### Step 1: Serial-to-Parallel Conversion

In the first step, we convert the length-$L$ symbol sequence into consecutive blocks containing $M$-symbols. If necessary, the last block is padded with 0s to fill it to length-$M$.

In Python, we accomplish S/P conversion by reshaping the (possibly padded) vector holding symbols into a matrix with trailing dimension equal to $M$. This choice is compatible with how the FFT routine in NumPy works.



In [7]:
def serial_to_parallel(syms, M):
    """Convert a vector of symbols to a matrix with trailing dimension M
    
    Inputs:
    * syms - vector of information symbols
    * M - block size

    Returns:
    matrix of dimensions (*,M)
    """
    L = len(syms)
    K = L % M      # L mod M

    if K != 0:
        syms = np.append(syms, np.zeros(M-K))  # zero-padding

    return np.reshape(syms, (-1, M))           # -1 means: figure out first dimension

In [29]:
# quick check
# make L random QPSK symbols
L = 13
syms = (1 - 2*np.random.randint(2, size=L)) + 1j*(1 - 2*np.random.randint(2, size=L))

# S/P with M symbols per block
M = 5
block_syms = serial_to_parallel(syms, M)
print("syms = ", syms)
print("block_syms = ", block_syms)

syms =  [-1.+1.j -1.+1.j  1.+1.j  1.-1.j -1.-1.j -1.-1.j  1.-1.j  1.+1.j  1.-1.j
  1.-1.j -1.-1.j -1.+1.j  1.-1.j]
block_syms =  [[-1.+1.j -1.+1.j  1.+1.j  1.-1.j -1.-1.j]
 [-1.-1.j  1.-1.j  1.+1.j  1.-1.j  1.-1.j]
 [-1.-1.j -1.+1.j  1.-1.j  0.+0.j  0.+0.j]]


### Step 2: Sub-carrier Mapping

In addition to what was discussed in class, a mechanism to select subcarriers is introduced. The goal is to select $M$ of the $N$ subcarriers that the (inverse) DFT provides. This facility can be used, for example, 

* to leave some subcarriers near the band edge unoccupied, or 
* to provide a flexible mechanism to control bandwidth, or
* to leave a subset of the subcarriers for use by other users; this facilitates multiple access referred to as OFDMA.

To indicate which subcarriers are occupied, we use a (boolean) vector of length $N$, where $N$ is the length of the FFT. This vector contains $M$ 1s to mark the subcarrier positions that will be used. The active subcarriers will be filled with the elements of a matrix of symbols.

Recall that the (I)FFT assumes subcarrier numbers from $0$ to $N-1$, i.e., from $f=0$ to $f=f_s$. Therefore, the band-edges are located in the center of the IFFT input.

In [31]:
def map_to_subcarriers(block_syms, active_sc):
    """Map a matrix of symbols to active subcarriers
    
    Inputs:
    * block_syms - matrix of symbols, dimension (*,M)
    * active_sc - length N vector that marks active subcarries with 1's 

    Returns:
    matrix of dimension (*,N)
    """
    # make an array to hold the result; figure out dimensions and data type
    N = len(active_sc)
    L = block_syms.shape[0]
    res = np.zeros((L,N), dtype=syms.dtype)

    res[:,np.equal(active_sc, 1)] = block_syms

    return res

In [32]:
# Quick check: we expand from 5 to 8 subcarriers
active = [1, 1, 1, 0, 0, 0, 1, 1]  # leave the middle subcarriers empty
block_sc = map_to_subcarriers(block_syms, active)

print("block_sc = ", block_sc)

block_sc =  [[-1.+1.j -1.+1.j  1.+1.j  0.+0.j  0.+0.j  0.+0.j  1.-1.j -1.-1.j]
 [-1.-1.j  1.-1.j  1.+1.j  0.+0.j  0.+0.j  0.+0.j  1.-1.j  1.-1.j]
 [-1.-1.j -1.+1.j  1.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]]


### Step 3: Inverse DFT

The heart of the OFDM transmitter is the inverse DFT, computed efficiently using an IFFT.

In [33]:
def idft(block_sc):
    """inverse DFT of matrix of symbols
    
    Inputs:
    block_sc - matrix of symbols (and empty subcarriers)

    Returns:
    matrix of the same size as the input
    """
    return np.fft.ifft(block_sc)

In [42]:
# Quick check
block_sig = idft(block_sc)

print("block_sig = ", block_sig)

block_sig =  [[-0.125     +0.125j     -0.72855339+0.125j     -0.625     +0.125j
   0.125     +0.125j      0.375     +0.125j     -0.02144661+0.125j
  -0.125     +0.125j      0.125     +0.125j    ]
 [ 0.375     -0.375j     -0.1982233 -0.3017767j -0.375     -0.125j
  -0.0517767 +0.0517767j -0.125     +0.125j     -0.5517767 +0.0517767j
  -0.375     -0.125j      0.3017767 -0.3017767j]
 [-0.125     -0.125j     -0.1767767 +0.j        -0.375     -0.125j
  -0.25      -0.4267767j  0.125     -0.375j      0.1767767 +0.j
  -0.125     +0.125j     -0.25      -0.0732233j]]


### Step 4: Insert Cyclic Prefix

To insert a cyclic prefix, the last $N_{CP}$ samples from each block are pre-pended to to the start of each block.

In [44]:
def insert_cyclic_prefix(block_sig, N_CP):
    """Insert cyclic prefix
    
    Inputs:
    block_sig - matrix of signal samples
    N_CP - length of cyclic prefix in samples

    Returns:
    signal matrix with cyclic prefix inserted; second dimension increases by N_CP
    """
    return np.append(block_sig[:,-N_CP:], block_sig, axis=1)

In [48]:
# Quick check
block_sig_cp = insert_cyclic_prefix(block_sig, 2)

print("block_sig_cp = ", block_sig_cp)
print("dimensions prior to CP: ", block_sig.shape)
print("dimensions after CP: ", block_sig_cp.shape)

block_sig_cp =  [[-0.125     +0.125j      0.125     +0.125j     -0.125     +0.125j
  -0.72855339+0.125j     -0.625     +0.125j      0.125     +0.125j
   0.375     +0.125j     -0.02144661+0.125j     -0.125     +0.125j
   0.125     +0.125j    ]
 [-0.375     -0.125j      0.3017767 -0.3017767j  0.375     -0.375j
  -0.1982233 -0.3017767j -0.375     -0.125j     -0.0517767 +0.0517767j
  -0.125     +0.125j     -0.5517767 +0.0517767j -0.375     -0.125j
   0.3017767 -0.3017767j]
 [-0.125     +0.125j     -0.25      -0.0732233j -0.125     -0.125j
  -0.1767767 +0.j        -0.375     -0.125j     -0.25      -0.4267767j
   0.125     -0.375j      0.1767767 +0.j        -0.125     +0.125j
  -0.25      -0.0732233j]]
dimensions prior to CP:  (3, 8)
dimensions after CP:  (3, 10)


### Step 5: Parallel-to-Serial Conversion

The final processing step is to concatenate the signal blocks and turn them into a vector.

In [52]:
def parallel_to_serial(block_sig):
    """convert a matrix of signal samples into a vector
    
    Inputs:
    block_sig - matrix of signal samples

    Returns:
    vector of signal samples
    """
    return np.reshape(block_sig, -1)  # -1 means: figure out how many samples

In [53]:
# quick check
sig = parallel_to_serial(block_sig_cp)

print("sig = ", sig)
print("signal has {:d} samples".format(len(sig)))

sig =  [-0.125     +0.125j      0.125     +0.125j     -0.125     +0.125j
 -0.72855339+0.125j     -0.625     +0.125j      0.125     +0.125j
  0.375     +0.125j     -0.02144661+0.125j     -0.125     +0.125j
  0.125     +0.125j     -0.375     -0.125j      0.3017767 -0.3017767j
  0.375     -0.375j     -0.1982233 -0.3017767j -0.375     -0.125j
 -0.0517767 +0.0517767j -0.125     +0.125j     -0.5517767 +0.0517767j
 -0.375     -0.125j      0.3017767 -0.3017767j -0.125     +0.125j
 -0.25      -0.0732233j -0.125     -0.125j     -0.1767767 +0.j
 -0.375     -0.125j     -0.25      -0.4267767j  0.125     -0.375j
  0.1767767 +0.j        -0.125     +0.125j     -0.25      -0.0732233j]
signal has 30 samples


### Trasmitter Object

For convenience, we define a class to represent an OFDM transmitter. The transmitter class wraps the above functions.

In [54]:
class OFDM_Transmitter():
    """OFDM transmitter class
    
    Parameters:
    N_CP - length of cyclic prefix
    active_sc - length N_FFT vector marking active subcarriers

    Example:
    # create a transmitter object
    tx = OFDM_Transmitter(N_FFT, N_CP, active_sc) 

    # pass symbols to tx object to generate signal
    sig = tx(syms)
    """
    def __init__(self, N_CP, active):
        self.N_CP = N_CP
        self.active = active

        self.M = np.sum(active)  # number of active subcarriers

    def __call__(self, syms):
        block_syms = serial_to_parallel(syms, self.M)
        block_sc = map_to_subcarriers(block_syms, self.active)
        block_sig = idft(block_sc)
        block_sig_cp = insert_cyclic_prefix(block_sig, self.N_CP)
        return parallel_to_serial(block_sig_cp)

In [56]:
tx = OFDM_Transmitter(2, active)
if np.allclose(sig, tx(syms)):
    print("Got same result")

Got same result
